<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 3 - 3 Hours </h1>
<h1 style="text-align:center">Long Short Term Memory (LSTM) for Language Modeling</h1>

<b> Student 1:</b> # Di HAO 
<b> Student 2:</b> # Chin-Te LIAO
 
 
In this Lab Session,  you will build and train a Recurrent Neural Network, based on Long Short-Term Memory (LSTM) units for next word prediction task. 

Answers and experiments should be made by groups of one or two students. Each group should fill and run appropriate notebook cells. 
Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an pdf document using print as PDF (Ctrl+P). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed by June 9th 2017.

Send you pdf file to benoit.huet@eurecom.fr and olfa.ben-ahmed@eurecom.fr using **[DeepLearning_lab3]** as Subject of your email.

#  Introduction

You will train a LSTM to predict the next word using a sample short story. The LSTM will learn to predict the next item of a sentence from the 3 previous items (given as input). Ponctuation marks are considered as dictionnary items so they can be predicted too. Figure 1 shows the LSTM and the process of next word prediction. 

<img src="lstm.png" height="370" width="370"> 


Each word (and ponctuation) from text sentences is encoded by a unique integer. The integer value corresponds to the index of the corresponding word (or punctuation mark) in the dictionnary. The network output is a one-hot-vector indicating the index of the predicted word in the reversed dictionnary (Section 1.2). For example if the prediction is 86, the predicted word will be "company". 



You will use a sample short story from Aesop’s Fables (http://www.taleswithmorals.com/) to train your model. 


<font size="3" face="verdana" > <i> "There was once a young Shepherd Boy who tended his sheep at the foot of a mountain near a dark forest.

It was rather lonely for him all day, so he thought upon a plan by which he could get a little company and some excitement.
He rushed down towards the village calling out "Wolf, Wolf," and the villagers came out to meet him, and some of them stopped with him for a considerable time.
This pleased the boy so much that a few days afterwards he tried the same trick, and again the villagers came to his help.
But shortly after this a Wolf actually did come out from the forest, and began to worry the sheep, and the boy of course cried out "Wolf, Wolf," still louder than before.
But this time the villagers, who had been fooled twice before, thought the boy was again deceiving them, and nobody stirred to come to his help.
So the Wolf made a good meal off the boy's flock, and when the boy complained, the wise man of the village said:
"A liar will not be believed, even when he speaks the truth."  "</i> </font>.    







Start by loading the necessary libraries and resetting the default computational graph. For more details about the rnn packages, we suggest you to take a look at https://www.tensorflow.org/api_guides/python/contrib.rnn

In [2]:
import numpy as np
import collections # used to build the dictionary
import random
import time
import pickle # may be used to save your model 
import matplotlib.pyplot as plt
#Import Tensorflow and rnn
import tensorflow as tf
from tensorflow.contrib import rnn  

# Target log path
logs_path = 'lstm_words'
writer = tf.summary.FileWriter(logs_path)

# Next-word prediction task

## Part 1: Data  preparation

### 1.1. Loading data

Load and split the text of our story

In [3]:
def load_data(filename):
    with open(filename) as f:
        data = f.readlines()
    data = [x.strip().lower() for x in data]
    data = [data[i].split() for i in range(len(data))]
    data = np.array(data)
    data = np.reshape(data, [-1, ])
    print(data)
    return data

#Run the cell 
train_file ='./story.txt'
train_data = load_data(train_file)
print("Loaded training data...")
print(len(train_data))

['there' 'was' 'once' 'a' 'young' 'shepherd' 'boy' 'who' 'tended' 'his'
 'sheep' 'at' 'the' 'foot' 'of' 'a' 'mountain' 'near' 'a' 'dark' 'forest'
 '.' 'it' 'was' 'rather' 'lonely' 'for' 'him' 'all' 'day' ',' 'so' 'he'
 'thought' 'upon' 'a' 'plan' 'by' 'which' 'he' 'could' 'get' 'a' 'little'
 'company' 'and' 'some' 'excitement' '.' 'he' 'rushed' 'down' 'towards'
 'the' 'village' 'calling' 'out' 'wolf' ',' 'wolf' ',' 'and' 'the'
 'villagers' 'came' 'out' 'to' 'meet' 'him' ',' 'and' 'some' 'of' 'them'
 'stopped' 'with' 'him' 'for' 'a' 'considerable' 'time' '.' 'this'
 'pleased' 'the' 'boy' 'so' 'much' 'that' 'a' 'few' 'days' 'afterwards'
 'he' 'tried' 'the' 'same' 'trick' ',' 'and' 'again' 'the' 'villagers'
 'came' 'to' 'his' 'help' '.' 'but' 'shortly' 'after' 'this' 'a' 'wolf'
 'actually' 'did' 'come' 'out' 'from' 'the' 'forest' ',' 'and' 'began' 'to'
 'worry' 'the' 'sheep,' 'and' 'the' 'boy' 'of' 'course' 'cried' 'out'
 'wolf' ',' 'wolf' ',' 'still' 'louder' 'than' 'before' '.' 'but' 't

### 1.2.Symbols encoding

The LSTM input's can only be numbers. A way to convert words (symbols or any items) to numbers is to assign a unique integer to each word. This process is often based on frequency of occurrence for efficient coding purpose.

Here, we define a function to build an indexed word dictionary (word->number). The "build_vocabulary" function builds both:

- Dictionary : used for encoding words to numbers for the LSTM inputs 
- Reverted dictionnary : used for decoding the outputs of the LSTM into words (and punctuation).

For example, in the story above, we have **113** individual words. The "build_vocabulary" function builds a dictionary with the following entries ['the': 0], [',': 1], ['company': 85],...


In [4]:
def build_vocabulary(words):
    count = collections.Counter(words).most_common()
    dic= dict()
    for word, _ in count:
        dic[word] = len(dic)
    reverse_dic= dict(zip(dic.values(), dic.keys()))
    return dic, reverse_dic


Run the cell below to display the vocabulary

In [5]:
dictionary, reverse_dictionary = build_vocabulary(train_data)
vocabulary_size= len(dictionary) 
print "Dictionary size (Vocabulary size) = ", vocabulary_size
print("\n")
print("Dictionary : \n")
print(dictionary)
print("\n")
print("Reverted Dictionary : \n" )
print(reverse_dictionary)

Dictionary size (Vocabulary size) =  113


Dictionary : 

{'all': 32, 'liar': 33, 'help': 17, 'cried': 34, 'course': 35, 'still': 36, 'pleased': 37, 'before': 18, 'excitement': 91, 'deceiving': 38, 'had': 39, 'young': 69, 'actually': 40, 'to': 6, 'villagers': 11, 'shepherd': 41, 'them': 19, 'lonely': 42, 'get': 44, 'dark': 45, 'not': 64, 'day': 47, 'did': 48, 'calling': 49, 'twice': 50, 'good': 51, 'stopped': 52, 'truth': 53, 'meal': 54, 'sheep,': 55, 'some': 20, 'tended': 56, 'louder': 57, 'flock': 58, 'out': 9, 'even': 59, 'trick': 60, 'said': 61, 'for': 21, 'be': 62, 'after': 63, 'come': 22, 'by': 65, 'boy': 7, 'of': 10, 'could': 66, 'days': 67, 'wolf': 5, 'afterwards': 68, ',': 1, 'down': 70, 'village': 23, 'sheep': 72, 'little': 73, 'from': 74, 'rushed': 75, 'there': 76, 'been': 77, '.': 4, 'few': 78, 'much': 79, "boy's": 80, ':': 81, 'was': 12, 'a': 2, 'him': 13, 'that': 83, 'company': 84, 'nobody': 85, 'but': 24, 'fooled': 86, 'with': 87, 'than': 43, 'he': 8, 'made': 89, 'wise':

## Part 2 : LSTM Model in TensorFlow

Since you have defined how the data will be modeled, you are now to develop an LSTM model to predict the word of following a sequence of 3 words. 

### 2.1. Model definition

Define a 2-layers LSTM model.  

For this use the following classes from the tensorflow.contrib library:

- rnn.BasicLSTMCell(number of hidden units) 
- rnn.static_rnn(rnn_cell, data, dtype=tf.float32)
- rnn.MultiRNNCell(,)


You may need some tensorflow functions (https://www.tensorflow.org/api_docs/python/tf/) :
- tf.split
- tf.reshape 
- ...




In [6]:
def lstm_model(x, w, b):
    
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x,n_input,1)
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])   
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    y=tf.matmul(outputs[-1], w['out']) + b['out']
    y=tf.nn.softmax(y)
    
    return y

Training Parameters and constants

In [7]:
# Training Parameters
learning_rate = 0.001
epochs = 50000
display_step = 1000
n_input = 3

#For each LSTM cell that you initialise, supply a value for the hidden dimension, number of units in LSTM cell
n_hidden = 64

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocabulary_size])

# LSTM  weights and biases
weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }


#build the model
pred = lstm_model(x, weights, biases)

Define the Loss/Cost and optimizer

In [8]:
# Loss and optimizer
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
#use RMSProp Optimizer
with tf.name_scope('RMSPropOptimizer'):
    optimizer =  tf.train.RMSPropOptimizer(learning_rate).minimize(cost)


# Model evaluatio    
with tf.name_scope('Accuracy'):
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    


We give you here the Test Function

In [9]:
#run the cell
def test(sentence, session, verbose=False):
    sentence = sentence.strip()
    words = sentence.split(' ')
    if len(words) != n_input:
        print("sentence length should be equel to", n_input, "!")
    try:
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        onehot_pred = session.run(pred, feed_dict={x: keys})
        onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval(session=sess))
        words.append(reverse_dictionary[onehot_pred_index])
        sentence = " ".join(words)
        #verbose=True
        if verbose:
            print(sentence)
        return reverse_dictionary[onehot_pred_index]
    except:
        print " ".join(["Word", words[i - n_input], "not in dictionary"])

## Part 3 : LSTM Training  

In the Training process, at each epoch, 3 words are taken from the training data, encoded to integer to form the input vector. The training labels are one-hot vector encoding the word that comes after the 3 inputs words. Display the loss and the training accuracy every 1000 iteration. Save the model at the end of training in the **lstm_model** folder

In [10]:
import random
# Initializing the variables
start_time = time.time()
#init = tf.global_variables_initializer()
model_saver = tf.train.Saver()

s_acc=0.0
s_loss=0.0
tf.summary.scalar("Loss", cost)
tf.summary.scalar("Accuracy", accuracy)

#merged_summary_op = tf.summary.merge_all()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Start Training")

for epoch in range(epochs):
        
        avg_cost = 0.
        random_number=random.randint(0,len(train_data)-n_input-1)
        words=train_data[random_number:random_number+n_input]
        symbols_inputs = [dictionary[str(words[i - n_input])] for i in range(n_input)]
        symbols_outputs = np.zeros([vocabulary_size], dtype=float)
        symbols_outputs[dictionary[str(train_data[random_number+n_input])]]=1
        keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        values = np.reshape(np.array(symbols_outputs), [-1, vocabulary_size])
        #keys = np.reshape(np.array(symbols_inputs), [-1, n_input, 1])
        _, acc, loss, onehot_pred = sess.run([optimizer,accuracy, cost,pred],
                                 feed_dict={x: keys, y: values})
        s_acc+=acc
        s_loss+=loss
        #onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
        # Write logs at every iteration
        #summary_writer.add_summary(summary, epoch)
        
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "Loss=", "{:.9f}".format(s_loss/epoch),"Acc=", "{:.9f}".format(s_acc/epoch))
            
print("Optimization Finished!")
# Save model
save_path = model_saver.save(sess, "/notebooks/liao/TP3/model_rnn.ckpt")
# Calculate accuracy
print ("Training Finished!")
##############################################
print("End Of training Finished!")
print("time: ",time.time() - start_time)
print("accuracy: ", s_acc/epochs)

print("For tensorboard visualisation run on command line.")
print("\ttensorboard --logdir=%s" % (logs_path))
print("and oint your web browser to the returned link")
##############################################
#save your model 
##############################################
print("Model saved")

Start Training
('Epoch: ', '1000', 'Loss=', '4.481712539', 'Acc=', '0.099099099')
('Epoch: ', '2000', 'Loss=', '4.173608594', 'Acc=', '0.104552276')
('Epoch: ', '3000', 'Loss=', '3.866063112', 'Acc=', '0.143714572')
('Epoch: ', '4000', 'Loss=', '3.541071392', 'Acc=', '0.193298325')
('Epoch: ', '5000', 'Loss=', '3.220579230', 'Acc=', '0.252850570')
('Epoch: ', '6000', 'Loss=', '2.912144536', 'Acc=', '0.315052509')
('Epoch: ', '7000', 'Loss=', '2.637457629', 'Acc=', '0.375625089')
('Epoch: ', '8000', 'Loss=', '2.394709129', 'Acc=', '0.430928866')
('Epoch: ', '9000', 'Loss=', '2.183823267', 'Acc=', '0.479719969')
('Epoch: ', '10000', 'Loss=', '2.005237252', 'Acc=', '0.520852085')
('Epoch: ', '11000', 'Loss=', '1.852588723', 'Acc=', '0.555777798')
('Epoch: ', '12000', 'Loss=', '1.719383728', 'Acc=', '0.586798900')
('Epoch: ', '13000', 'Loss=', '1.603255668', 'Acc=', '0.614662666')
('Epoch: ', '14000', 'Loss=', '1.502158612', 'Acc=', '0.638545610')
('Epoch: ', '15000', 'Loss=', '1.413582980

ValueError: Parent directory of /notebooks/liao/TP3/model_rnn.ckpt doesn't exist, can't save.

## Part 4 : Test your model 

### 4.1. Next word prediction

Load your model (using the model_saved variable given in the training session) and test the sentences :
- 'get a little' 
- 'nobody tried to'
- Try with other sentences using words from the stroy's vocabulary. 

In [10]:
#Your implementation goes here 
test('get a little',sess,verbose=True)
test('nobody tried to',sess,verbose=True)
test('boy of course',sess,verbose=True)

get a little company
nobody tried to come
boy of course cried


'cried'

### 4.2. More fun with the Fable Writer !

You will use the RNN/LSTM model learned in the previous question to create a
new story/fable.
For this you will choose 3 words from the dictionary which will start your
story and initialize your network. Using those 3 words the RNN will generate
the next word or the story. Using the last 3 words (the newly predicted one
and the last 2 from the input) you will use the network to predict the 5
word of the story.. and so on until your story is 5 sentence long. 
Make a point at the end of your story. 
To implement that, you will use the test function. 

In [11]:

#Your implementation goes here 

def generate_story(first_3_words,i=0):
    print test(first_3_words,sess),
    a,b,c = first_3_words.split(" ")
    new_string = b +" "+ c +" "+ test(first_3_words,sess)
    if test(first_3_words,sess)==".":
        i=i+1
        if i>4:
            return 0
    return generate_story(new_string,i)

n_input=3   
a=generate_story('was once a')

young shepherd boy who tended his sheep at the foot of a mountain near a dark meal off the boy's flock , and when the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a dark meal off the boy's flock , and when the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a dark meal off the boy's flock , and when the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a dark meal off the boy's flock , and when the villagers came out to meet him , and some of them stopped with him for a considerable time . this pleased the boy so much that a dark meal off the boy's flock , and when the villagers came out to meet him , and some of them stopped with him for a considerable time .


### 4.3. Play with number of inputs

The number of input in our example is 3, see what happens when you use other number (1 and 5)

Your answer goes here




In [13]:
# Training Parameters

# n_input = 4

# x = tf.placeholder("float", [None, n_input, 1])

# weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size]))}
# biases = {'out': tf.Variable(tf.random_normal([vocabulary_size])) }
# #build the model
# pred = lstm_model(x, weights, biases)
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

test('who tended his sheep',sess,verbose=True)

ValueError: Attempt to have a second RNNCell use the weights of a variable scope that already has weights: 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell'; and the cell was not constructed as BasicLSTMCell(..., reuse=True).  To share the weights of an RNNCell, simply reuse it in your second calculation, or create a new one with the argument reuse=True.